In [ ]:
import panqec
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import hashlib
import json
import binascii
from panqec.utils import NumpyEncoder, hash_json
from panqec.statmech.rbim2d import (
    RandomBondIsingModel2D, Magnetization,
    Rbim2DIidDisorder
)
from panqec.statmech.controllers import SimpleController
from panqec.statmech.analysis import SimpleAnalysis, load_analysis
from panqec.statmech.plots import (
    get_wilson_loop_data, get_wilson_loop_critical_temperatures, 
    get_wilson_loop_ansatz_parameters, 
    plot_phase_diagram, plot_wilson_loops, plot_ansatz_wilson_loops
)
import uuid
import pandas as pd
import cProfile
from pstats import Stats
import os
import psutil
from itertools import product

## Load data

In [ ]:
data_dir = '../temp/statmech/test_7/'

In [ ]:
analysis = load_analysis(data_dir)

In [ ]:
estimates = analysis.estimates

In [ ]:
max_tau = estimates['tau'].max()

## Plot single observable

In [ ]:
label = 'CorrelationRatio'
for p_val in estimates['p'].unique():
    for L_x_val in estimates['L_x'].unique():
        estimates_filt = estimates[
            (estimates['p'] == p_val)
            & (estimates['tau'] == max_tau)
            & (estimates['L_x'] == L_x_val)
        ][['temperature', f'{label}_estimate', f'{label}_uncertainty']]
        y_values = np.real(estimates_filt[f'{label}_estimate'])
        y_values[y_values == 0] = np.nan
        y_err = np.real(estimates_filt[f'{label}_uncertainty'])
        plt.errorbar(
            estimates_filt['temperature'],
            y_values,
            yerr=y_err,
            fmt='o-',
            capsize=5,
            label=f'L = {L_x_val}'
        )
    plt.ylabel(r'Correlation Ratio $\xi/L$')
    plt.xlabel('Temperature $T$')
    plt.title(f'p = {p_val}')
    plt.yscale('log')
    plt.legend()
    plt.show()

## Plot multiple observable

In [ ]:
observable_list = [
    'Energy', 'SpecificHeat', 'Magnetization', 'Magnetization_2',
    'Susceptibility0', 'Susceptibilitykmin', 'CorrelationRatio',
    'Binder'
]
for label in observable_list:
    for p_val in estimates['p'].unique():
        for L_x_val in estimates['L_x'].unique():
            estimates_filt = estimates[
                (estimates['p'] == p_val)
                & (estimates['tau'] == max_tau)
                & (estimates['L_x'] == L_x_val)
            ][['temperature', f'{label}_estimate', f'{label}_uncertainty']]
            # display(estimates_filt[['temperature']])
            y_values = np.real(estimates_filt[f'{label}_estimate'])
            y_values[y_values == 0] = np.nan
            y_err = np.real(estimates_filt[f'{label}_uncertainty'])
            plt.errorbar(
                estimates_filt['temperature'],
                y_values,
                yerr=y_err,
                fmt='o-',
                capsize=5,
                label=f'L = {L_x_val}'
            )
        plt.ylabel(label)
        plt.xlabel('temperature')
        plt.title(f'p = {p_val}')
        plt.legend()
        plt.show()

## Plots for Wilson loops

### Wilson loop curves

In [ ]:
plot_wilson_loops(estimates, max_tau=max_tau-1)

### Ansatz parameters

In [ ]:
plot_ansatz_wilson_loops(estimates, max_tau=max_tau-1, start=4)

### Phase diagram

In [ ]:
critical_temperatures = get_wilson_loop_critical_temperatures(estimates, max_tau=max_tau-1, start=4)

In [ ]:
probas = estimates['p'].unique()
plot_phase_diagram(probas, critical_temperatures)